In [1]:
#Load final training df as generated from notebook 1.4
import pandas as pd

df = pd.read_csv("dset_augmented.csv") #assume df within repository and not in local data repositories

#todo name cols accordingly
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.121850,0.002779,2.820640,0.362703,0.299531,1.064317,0.026883,0.283242,-47.188390,0.114930,...,0.004362,0.004121,0.004833,0.002142,0.003436,0.004189,0.004279,0.003408,0.035714,healthy
1,0.202018,0.002003,2.689907,0.257321,0.165729,1.256261,0.071003,0.366933,-62.221158,0.306251,...,0.005076,0.005123,0.005872,0.002695,0.004685,0.005683,0.005179,0.004893,0.030607,healthy
2,0.271211,0.002006,2.717560,0.439047,0.288369,1.790830,0.008095,0.576133,-46.771765,0.152389,...,0.005709,0.005680,0.007379,0.002932,0.004767,0.005241,0.005232,0.003955,0.026166,healthy
3,0.397790,0.005861,2.828297,0.445612,0.254581,1.517740,0.018675,0.573076,-38.613659,0.790263,...,0.009088,0.014426,0.017315,0.002497,0.017967,0.008387,0.009203,0.003781,0.018469,healthy
4,0.408471,0.005529,2.808983,0.457015,0.263126,1.606012,0.017185,0.610265,-39.037931,0.820648,...,0.006608,0.008596,0.012481,0.001461,0.014940,0.009628,0.008017,0.003102,0.014015,healthy


In [2]:
#convert df to input output X Y 
from sklearn.model_selection import train_test_split 
from sklearn import svm

X = df.iloc[: , :-1] #everything except last column
y = df.iloc[:, 34] #last column
print(" shape", X.shape)
print("y shape", y.shape)


 shape (11835, 34)
y shape (11835,)


In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#apply pca to reduce feature space 
pca = PCA(n_components=0.95) #0.95 variance
X_pca = pca.fit_transform(X_scaled)

print(f"Number of components chosen: {pca.n_components_}")

Number of components chosen: 23


In [21]:
#TODO more sophisticated train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# 1. Load your data
# df = pd.read_csv('your_dataset.csv')
# X = df.drop('target_column', axis=1)
# y = df['target_column']

# 2. Scaling is CRITICAL for SVM complexity
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. Define the Complex Model Search
# RBF kernel handles non-linear patterns by mapping to infinite dimensions
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],          # Penalty for misclassification
    'gamma': [1, 0.1, 0.01, 0.001],        # Kernel coefficient (influence radius)
    'kernel': ['rbf']                      # The 'complex' kernel
}

# 4. Use GridSearchCV to find the best version of the model
grid = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=2, cv=5)
grid.fit(X_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   4.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   4.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   4.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   4.1s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   4.1s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   4.1s
[CV] END ......................C=0.1, gamma=0.0

,estimator,SVC()
,param_grid,"{'C': [0.1, 1, ...], 'gamma': [1, 0.1, ...], 'kernel': ['rbf']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,100


In [24]:



# 5. Final Model
best_model = grid.best_estimator_
y_predict = best_model.predict(X_test)

In [25]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


print("\nClassification Report:")
print(classification_report(y_test, y_predict))


print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_predict))


Classification Report:
              precision    recall  f1-score   support

    COVID-19       0.68      0.74      0.71       778
     healthy       0.70      0.69      0.70       800
 symptomatic       0.72      0.67      0.69       789

    accuracy                           0.70      2367
   macro avg       0.70      0.70      0.70      2367
weighted avg       0.70      0.70      0.70      2367


Confusion Matrix:
[[574 102 102]
 [143 551 106]
 [129 130 530]]
